In [42]:
from trajectory_utils import filter_joint_states
import json

input_file = "split_lego_both.json"  # Update the path as needed
with open(input_file, "r") as file:
    js = json.load(file)

In [45]:
filtered_js = filter_joint_states(js, 0.01)
print(len(filtered_js))

for i, _ in enumerate(filtered_js):
    if i < 100:
        js[i]['gripper_state'] = ('open_left', 'open_right')
    else:
        js[i]['gripper_state'] = ('close_left', 'open_right')

with open("split_lego_both.json", "w") as file:
    json.dump(filtered_js, file, indent=4)

211


In [40]:
len(filtered_js)

211

In [1]:
from trajectory_utils import filter_joint_states
import json

input_file = "split_lego_both.json"  # Update the path as needed

with open(input_file, "r") as file:
    js = json.load(file)
    
filtered_joint_states = filter_joint_states(js, 0.001)
len(filtered_joint_states)

211

In [2]:
from ikSolver import IKSolver
from get_fk import GetFK
from trajectory_utils import filter_joint_states
from replay_trajectory import dict_to_joint_state

gfk_left = GetFK('gripper_l_base', 'world')
gfk_right = GetFK('gripper_r_base', 'world')

msgs = [dict_to_joint_state(filtered_joint_state) for filtered_joint_state in filtered_joint_states]

eef_poses_left = [gfk_left.get_fk(msg) for msg in msgs]
eef_poses_right = [gfk_right.get_fk(msg) for msg in msgs]

In [3]:
left_eef = "gripper_l_base"
right_eef = "gripper_r_base"

ik_solver_left = IKSolver(group_name="left_arm", ik_link_name=left_eef)
ik_solver_right = IKSolver(group_name="right_arm", ik_link_name=right_eef)

waypoints_left = [eef_pose.pose_stamped[0].pose for eef_pose in eef_poses_left]
waypoints_right = [eef_pose.pose_stamped[0].pose for eef_pose in eef_poses_right]
# Example coordinates and orientation (Euler angles in radians)
# for eef_pose_left in waypoints_left:
#     ik_left = ik_solver_left.get_ik(eef_pose_left).solution.joint_state.position


[ WARN] [1706900573.268136034]: IK plugin for group 'left_arm' relies on deprecated API. Please implement initialize(RobotModel, ...).
[ WARN] [1706900573.273064414]: IK plugin for group 'right_arm' relies on deprecated API. Please implement initialize(RobotModel, ...).


[ INFO] [1706900573.139605928]: Loading robot model 'yumi'...
[ INFO] [1706900573.270880324]: IK Using joint yumi_link_1_l -2.94088 2.94088
[ INFO] [1706900573.270899531]: IK Using joint yumi_link_2_l -2.50455 0.759218
[ INFO] [1706900573.270910775]: IK Using joint yumi_link_3_l -2.94088 2.94088
[ INFO] [1706900573.270921670]: IK Using joint yumi_link_4_l -2.15548 1.39626
[ INFO] [1706900573.270930261]: IK Using joint yumi_link_5_l -5.06145 5.06145
[ INFO] [1706900573.270940109]: IK Using joint yumi_link_6_l -1.53589 2.40855
[ INFO] [1706900573.270949537]: IK Using joint yumi_link_7_l -3.9968 3.9968
[ INFO] [1706900573.270963506]: Looking in common namespaces for param name: left_arm/position_only_ik
[ INFO] [1706900573.272039417]: Looking in common namespaces for param name: left_arm/solve_type
[ INFO] [1706900573.273026560]: Using solve type Manipulation2
[ INFO] [1706900573.275314853]: IK Using joint yumi_link_1_r -2.94088 2.94088
[ INFO] [1706900573.275328612]: IK Using joint yumi_

[ WARN] [1706900573.420716480]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420759852]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420782829]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420808461]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900573.419327 according to authority unknown_publisher
[ WARN] [1706900573.420842404]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900573.419327 according t

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def solve_ik(ik_solver, pose):
    return ik_solver.get_ik(pose).solution.joint_state.position

# Using ThreadPoolExecutor to parallelize IK solving
with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
    futures_left = {executor.submit(solve_ik, ik_solver_left, pose): pose for pose in waypoints_left}
    futures_right = {executor.submit(solve_ik, ik_solver_right, pose): pose for pose in waypoints_right}

    results_left = [future.result() for future in as_completed(futures_left)]
    results_right = [future.result() for future in as_completed(futures_right)]


[ WARN] [1706900574.420638194]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900574.419337 according to authority unknown_publisher
[ WARN] [1706900574.420676956]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900574.419337 according to authority unknown_publisher
[ WARN] [1706900574.420706569]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900574.419337 according to authority unknown_publisher
[ WARN] [1706900574.420728499]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900574.419337 according to authority unknown_publisher
[ WARN] [1706900574.420749382]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900574.419337 according t

[INFO] [1706900574.645489]: IK successfully computed
[INFO] [1706900574.745831]: IK successfully computed
[INFO] [1706900574.846231]: IK successfully computed


[ WARN] [1706900574.920967396]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900574.919355 according to authority unknown_publisher
[ WARN] [1706900574.921009790]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900574.919355 according to authority unknown_publisher
[ WARN] [1706900574.921030323]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900574.919355 according to authority unknown_publisher
[ WARN] [1706900574.921048971]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900574.919355 according to authority unknown_publisher
[ WARN] [1706900574.921066501]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900574.919355 according t

[INFO] [1706900574.946778]: IK successfully computed
[INFO] [1706900575.047158]: IK successfully computed
[INFO] [1706900575.147542]: IK successfully computed


[ WARN] [1706900575.121074982]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_7_r (parent yumi_link_6_r) at time 1706900575.119323 according to authority unknown_publisher
[ WARN] [1706900575.121094118]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_3_l (parent yumi_link_2_l) at time 1706900575.119323 according to authority unknown_publisher
[ WARN] [1706900575.121106410]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_3_r (parent yumi_link_2_r) at time 1706900575.119323 according to authority unknown_publisher
[ WARN] [1706900575.220803277]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900575.219364 according to authority unknown_publisher
[ WARN] [1706900575.220849373]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900575.219364 according to author

...

[INFO] [1706900616.513347]: IK successfully computed
[INFO] [1706900616.613857]: IK successfully computed
[INFO] [1706900616.714248]: IK successfully computed


[ WARN] [1706900616.620929955]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900616.619352 according to authority unknown_publisher
[ WARN] [1706900616.620963340]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900616.619352 according to authority unknown_publisher
[ WARN] [1706900616.620981429]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900616.619352 according to authority unknown_publisher
[ WARN] [1706900616.620999029]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900616.619352 according to authority unknown_publisher
[ WARN] [1706900616.621019213]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900616.619352 according t

[INFO] [1706900616.814610]: IK successfully computed
[INFO] [1706900616.914997]: IK successfully computed


[ WARN] [1706900616.821006538]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_6_l (parent yumi_link_5_l) at time 1706900616.819330 according to authority unknown_publisher
[ WARN] [1706900616.821022881]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_6_r (parent yumi_link_5_r) at time 1706900616.819330 according to authority unknown_publisher
[ WARN] [1706900616.821045160]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_7_l (parent yumi_link_6_l) at time 1706900616.819330 according to authority unknown_publisher
[ WARN] [1706900616.821063319]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_7_r (parent yumi_link_6_r) at time 1706900616.819330 according to authority unknown_publisher
[ WARN] [1706900616.821081548]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_3_l (parent yumi_link_2_l) at time 1706900616.819330 according to authority unknown_

[ WARN] [1706900616.920734767]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_r (parent gripper_l_base) at time 1706900616.919344 according to authority unknown_publisher
[ WARN] [1706900616.920772900]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_l_finger_l (parent gripper_l_base) at time 1706900616.919344 according to authority unknown_publisher
[ WARN] [1706900616.920789523]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_r (parent gripper_r_base) at time 1706900616.919344 according to authority unknown_publisher
[ WARN] [1706900616.920807192]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame gripper_r_finger_l (parent gripper_r_base) at time 1706900616.919344 according to authority unknown_publisher
[ WARN] [1706900616.920832545]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame yumi_link_1_l (parent yumi_body) at time 1706900616.919344 according t